In [1]:
!apt-get -qq update > /tmp/apt.out
!apt-get install -y -qq openjdk-11-jdk-headless
!(wget -q --show-progress -nc https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz)
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
try:
  import pyspark, findspark, delta
except:
  %pip install -q --upgrade pyspark==3.2.1
  %pip install -q findspark
  %pip install -q delta

spark-3.2.1-bin-had 100%[===================>] 287.03M  12.7MB/s    in 24s     
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 kB 19.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
!pip install gcsfs

In [3]:
!(wget -O /content/spark-3.2.1-bin-hadoop3.2/jars/gcs-connector-hadoop2-latest.jar  -q https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3.2-latest.jar)

In [4]:
import findspark
import pyspark
import os
from pyspark.sql.functions import *

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

findspark.init()
MAX_MEMORY="8g"

spark = (pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.executor.memory", MAX_MEMORY)
    .config("spark.driver.memory", MAX_MEMORY)
    .getOrCreate()
    )

spark

In [ ]:
#spark.stop()

In [5]:
!wget https://storage.googleapis.com/bigdata_training/Titanic.parquet

--2024-03-13 19:39:51--  https://storage.googleapis.com/bigdata_training/Titanic.parquet
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.188.207, 64.233.189.207, 108.177.97.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.188.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40013 (39K) [application/octet-stream]
Saving to: ‘Titanic.parquet’

Titanic.parquet     100%[===================>]  39.08K  --.-KB/s    in 0.05s   

2024-03-13 19:39:53 (720 KB/s) - ‘Titanic.parquet’ saved [40013/40013]



In [6]:
df_passengers = spark.read.parquet("Titanic.parquet")
df_passengers.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [15]:
#Gemiden kurtulma durumuna göre ödenen en yüksek ve en düşük bilet fiyatını bulunuz.
df_passengers.groupBy("Survived") \
    .agg(max("Fare").alias("Max_Bilet_Fiyatı"), min("Fare").alias("Min_Bilet_Fiyatı")) \
    .withColumn("Durum", when(col("Survived") == 1, "Hayatta Kalanlar").otherwise("Hayatta Kalamayanlar")) \
    .select("Durum", "Max_Bilet_Fiyatı", "Min_Bilet_Fiyatı") \
    .show()



+--------------------+----------------+----------------+
|               Durum|Max_Bilet_Fiyatı|Min_Bilet_Fiyatı|
+--------------------+----------------+----------------+
|Hayatta Kalamayanlar|           263.0|             0.0|
|    Hayatta Kalanlar|        512.3292|             0.0|
+--------------------+----------------+----------------+



In [13]:
#Yolcuların(18 yaş ve altı çocuklar hariç) cinsiyete göre ortalama yaşlarını bulunuz.
df_passengers.where(col("Age") > 18) \
             .groupBy("Sex") \
             .agg(avg("Age").alias("Ortalama_Yaş")) \
.show()

+------+------------------+
|   Sex|      Ortalama_Yaş|
+------+------------------+
|female| 33.90673575129534|
|  male|34.480366492146594|
+------+------------------+



In [7]:
#Aynı kabinde yolculuk yapan erkek ve kadınların eşleşmesi yapılması için aşağıdaki sorgu yazılmışmış fakat sorguda ufak bir eksiklik var, eksik olan parçayı tamamlayınız.
df_passengers.alias("m") \
.join(df_passengers.alias("f"),
      on=((col("m.Cabin")==col("f.Cabin"))),
      how="inner") \
.where(col("f.Sex")=="male") \
.where(col("m.Sex")=="female") \
.select(col("m.Cabin"),col("f.Name").alias("Erkek Yolcu"),col("m.Name").alias("Kadın Yolcu")).show(100,200)

+-----------+------------------------------------------+----------------------------------------------------------------------------------+
|      Cabin|                               Erkek Yolcu|                                                                       Kadın Yolcu|
+-----------+------------------------------------------+----------------------------------------------------------------------------------+
|       C123|               Futrelle, Mr. Jacques Heath|                                      Futrelle, Mrs. Jacques Heath (Lily May Peel)|
|        D33|                 Harper, Mr. Henry Sleeper|                                          Harper, Mrs. Henry Sleeper (Myna Haxtun)|
|C23 C25 C27|                         Fortune, Mr. Mark|                                                        Fortune, Miss. Mabel Helen|
|C23 C25 C27|            Fortune, Mr. Charles Alexander|                                                        Fortune, Miss. Mabel Helen|
|         C2|       

In [22]:
#veriye yolcunun yaşına göre belirlenecek yeni bir kolon eklenmek isteniyor(AgeCategory). Kolon içeriğinde;
#0-10 yaş arası yolcular -> Çocuk
#11-18 yaş arası yolcular -> Genç
#19 ve üstü yolcular -> Yetişkin
#bilgisi yazılması isteniyor. Bu yeni kolonu ekleyiniz.
df_passengers.withColumn("AgeCategory", when(col("Age").between(0, 10), "Çocuk").when(col("Age").between(11, 18), "Genç").otherwise("Yetişkin")).show()


+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|AgeCategory|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|   Yetişkin|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|   Yetişkin|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|   Yetişkin|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|   Yetişkin|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|      

In [24]:
#spark'ta bir dataframe sorgu içnide de kullanılabilir. AgeCategory(bir üst sorunın çıktısı olan kolon) kolonuna göre yolcu sayısını veren SQL sorgunu düzenleyiniz.
df_passengers.createOrReplaceTempView("passengers")
spark.sql("select AgeCategory, count(*) as PassengerCount from (select *, case when age <= 10 then 'çocuk' when age <= 18 then 'genç' else 'yetişkin' end as AgeCategory from passengers) as t group by AgeCategory").show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|AgeCategory|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|   yetişkin|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|   yetişkin|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|   yetişkin|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|   yetişkin|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|      